In [1]:
from deep_orderbook.config import ReplayConfig, ShaperConfig
from deep_orderbook.shaper import iter_shapes_t2l
from deep_orderbook.visu import Visualizer

In [2]:
replay_conf_3 = ReplayConfig(
    markets=["ETH-USD"],#, "BTC-USD", "ETH-BTC"],
    data_dir='data',
    date_regexp='2024-1*T16',
    max_samples=-1,
    every="100ms",
)
shaper_config = ShaperConfig(
    only_full_arrays=False,
    zoom_frac = 0.0005,
    look_ahead_side_bips = 2,
)

In [ ]:
vis = Visualizer()
async def rolling_plot(config: ReplayConfig):
    async for books_array, t2l_array, pxar in iter_shapes_t2l(
        replay_config=config.but(every="200ms"),
        shaper_config=shaper_config.but(zoom_frac=0.0004, look_ahead_side_bips=1)
    ):
        vis.update(books_z_data=books_array, level_reach_z_data=t2l_array, bidask=pxar)

await rolling_plot(config=replay_conf_3.but(
        max_samples=100, 
        # skip_until_time="23:15",
    )
)